In [1]:
from surprise import accuracy, Reader, Dataset, SVD
from surprise.model_selection import cross_validate

In [2]:
import pandas as pd
import pathlib
DATA_DIR = pathlib.Path().resolve().parent / "data"
print(DATA_DIR)
DATA_DIR.exists()

C:\Users\SungWook\Desktop\Recom\PJT001\src\data


True

In [3]:
dataset = DATA_DIR / 'ratings_small.csv'
dataset.exists()

True

In [5]:
df = pd.read_csv(dataset)
df['rating'].dropna(inplace=True) # 빈 등급을 삭제
df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
df.rating.max(), df.rating.min()

(5.0, 0.5)

In [7]:
reader = Reader(rating_scale=(0.5, 5)) # 0.5 부터 5까지 평가를 할 수 있다
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)

In [8]:
algo = SVD(verbose=True, n_epochs=20) # SVD 라는 기계학습 알고리즘의 한 유형
# 알고리즘을 제곱 평균의 데이터와 비교하는 것
# 예측을 실행하는 최적의 방법을 찾고있음
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=4, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing

{'test_rmse': array([0.89981637, 0.89868308, 0.90000079, 0.90214414]),
 'test_mae': array([0.69410748, 0.69360805, 0.69319711, 0.69318848]),
 'fit_time': (0.7589805126190186,
  0.9561264514923096,
  0.8101842403411865,
  0.8455460071563721),
 'test_time': (0.1294567584991455,
  0.27551841735839844,
  0.11489033699035645,
  0.09903383255004883)}

In [10]:
# 훈련
trainset = data.build_full_trainset()
algo.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [13]:
testset = trainset.build_testset()
predictions = algo.test(testset)

accuracy.rmse(predictions, verbose=True)
accuracy.mae(predictions, verbose=True)

RMSE: 0.6433
MAE:  0.4978


0.49780351122821664

In [17]:
sample_row = df.sample(n=1)
userId = sample_row['userId'].values[0]
movieId = sample_row['movieId'].values[0]
print(userId, movieId)

388 2640


In [18]:
pred = algo.predict(uid=userId, iid=movieId)
pred.est

3.6523289869639424

In [19]:
import pickle

In [20]:
algo_data = {"model":algo}
with open('model.pkl', 'wb') as f:
    pickle.dump(algo_data, f)

In [21]:
model_algo = None
with open('model.pkl', 'rb') as f:
    model_data_loaded = pickle.load(f)
    model_algo = model_data_loaded.get('model')
    

In [22]:
model_algo.predict(uid=userId, iid=movieId)

Prediction(uid=388, iid=2640, r_ui=None, est=3.6523289869639424, details={'was_impossible': False})

In [23]:
sample_rows = df.sample(n=10).to_dict('records')
for row in sample_rows:
    userId = row['userId']
    movieId = row['movieId']
    pred = algo.predict(uid=userId, iid=movieId).est
    print(userId, movieId, pred)

264 2671 3.7191085964195785
615 72998 3.824036715304696
143 3949 3.723530577240503
592 1722 4.159983859664922
321 707 3.1488174783138496
134 1094 4.150559492151502
666 587 2.765410594637221
23 1722 2.998023965974821
523 49772 3.8189168971575094
664 65642 3.9167403794237763
